This jupyter notebook has been created to show [Folium] library capabilities with maps. 

Folium makes it easy to visualize data that’s been manipulated in Python on an interactive [Leaflet.js] map.

It is possible to interact with this jupyter notebook using [nbviewer] in the following link:

http://nbviewer.jupyter.org/github/alvertogit/datascience/blob/master/Map%20examples%20with%20Folium.ipynb

[Folium]: https://github.com/python-visualization/folium
[Leaflet.js]: https://leafletjs.com/
[nbviewer]: http://nbviewer.jupyter.org/

In [1]:
import folium

Countries selection on World Map

In [ ]:
# world countries boundaries json file
world_countries = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"

import urllib.request, json 
with urllib.request.urlopen(world_countries) as url:
    geo_json_data = json.loads(url.read().decode())

    
def country_color_function(feature):
    """Countries which id start with 'A' set to green and others to grey."""
    if str(feature['id']).startswith('A'):
        return '#00f000'
    else:
        return '#1f0000'
    
world_map = folium.Map(
    location=[0.0, 0.0],
    tiles='cartodbpositron',
    zoom_start=2
)

folium.GeoJson(
    world_countries,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': country_color_function(feature),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(world_map)


world_map

Markers on Map

In [2]:
from folium.plugins import MarkerCluster

madrid_map = folium.Map(location=[40.4169, -3.7033],
                        tiles = "stamenterrain",
                        zoom_start = 11)

# create a marker cluster added to the map
marker_cluster = MarkerCluster().add_to(madrid_map)

# add markers to marker cluster
folium.Marker(
    location=[40.47, -3.57],
    popup='Adolfo Suarez Madrid Barajas Airport.',
    icon=folium.Icon(color='green', icon='plane'),
).add_to(marker_cluster)


folium.Marker(
    location=[40.406, -3.69],
    popup='Madrid Puerta de Atocha Station.',
    icon=folium.Icon(color='blue', icon_color='white', icon='ok-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[40.5, -3.66],
    popup='Home.',
    icon=folium.Icon(color='red', icon='home'),
).add_to(marker_cluster)

folium.Marker(
    location=[40.44, -3.73],
    popup='Museum.',
    icon=folium.Icon(color='red', icon='remove-sign'),
).add_to(marker_cluster)

# add table to popup

import pandas as pd

df = pd.DataFrame(data=[['09:00 to 19:00', '10:00 to 17:00'], ['10:00 to 18:00', '10:00 to 15:00']], 
                  columns=['High Season Opening Hours', 'Low Season Opening Hours'], 
                  index=['Monday to Friday','Weekends & Holidays'])

html = df.to_html(classes='table table-striped table-hover table-condensed table-responsive')

popup_table = folium.Popup(html)

folium.Marker(
    location=[40.485, -3.74],
    popup=popup_table,
    icon=None,
).add_to(marker_cluster)

madrid_map

Heat Map generated with random data

In [3]:
import numpy as np

data = (np.random.normal(size=(50, 3)) *
        np.array([[5, 5, 5]]) +
        np.array([[47, 20, 1]])).tolist()

In [4]:
from folium.plugins import HeatMap

europe_map = folium.Map([47, 2], tiles='stamentoner', zoom_start=4)

HeatMap(data).add_to(europe_map)

europe_map

Lines and circles on Map

In [5]:
coordinates_seville_paris = [
    [48.858093, 2.294694],
    [41.390205, 2.154007],
    [40.4169, -3.7033],
    [37.392529, -5.994072]
]

coordinates_paris_brussels = [
    [50.8333, 4.35],
    [48.858093, 2.294694]
]

coordinates_brussels_berlin = [
    [52.5186, 13.4081],
    [50.8333, 4.35]
]

coordinates_paris_london = [
    [51.5072, -0.1275],
    [50.9475, 1.91],
    [48.858093, 2.294694]
]

# create the map
lines_map = folium.Map(location=[46, 0], zoom_start=5)

# create feature group
fg = folium.FeatureGroup(name="My Lines Map")

# add the lines and circles to the feature group
my_PolyLine=folium.PolyLine(locations=coordinates_seville_paris,weight=5)
fg.add_child(my_PolyLine)
for coordinates in coordinates_seville_paris:
    my_Circle=folium.CircleMarker(location=coordinates,
                            radius = 6,
                            popup="station",
                            fill=True,
                            fill_color='white',
                            color = 'grey',
                            fill_opacity=0.7)
    fg.add_child(my_Circle)

my_PolyLine=folium.PolyLine(color='red',locations=coordinates_paris_brussels,weight=5)
fg.add_child(my_PolyLine)

for coordinates in coordinates_paris_brussels:
    my_Circle=folium.CircleMarker(location=coordinates,
                            radius = 6,
                            popup="station",
                            fill=True,
                            fill_color='white',
                            color = 'grey',
                            fill_opacity=0.7)
    fg.add_child(my_Circle)

my_PolyLine=folium.PolyLine(color='green',locations=coordinates_brussels_berlin,weight=5)
fg.add_child(my_PolyLine)

for coordinates in coordinates_brussels_berlin:
    my_Circle=folium.CircleMarker(location=coordinates,
                            radius = 6,
                            popup="station",
                            fill=True,
                            fill_color='white',
                            color = 'grey',
                            fill_opacity=0.7)
    fg.add_child(my_Circle)

my_PolyLine=folium.PolyLine(color='orange',locations=coordinates_paris_london,weight=5)
fg.add_child(my_PolyLine)

for coordinates in coordinates_paris_london:
    my_Circle=folium.CircleMarker(location=coordinates,
                            radius = 6,
                            popup="Station",
                            fill=True,
                            fill_color='white',
                            color = 'grey',
                            fill_opacity=0.7)
    fg.add_child(my_Circle)
    
# add the feature group to the map
lines_map.add_child(fg)